In [20]:
import pandas as pd
import nltk

In [21]:
#Importing the csv file
news=pd.read_csv('news.csv')

In [22]:
#Exploratory Data Analysis
news.head()

,Category,News
0,ca,"The company put in an aggressive bid of Rs 6,0..."
1,notca,Reliance Capital today announced the appointme...
2,notca,Reports have emerged that Deloitte - the statu...
3,ca,HG Infra Engineering today fixed a price band ...
4,ca,US private equity firm Blackstone Group LP is ...


In [23]:
#Exploratory Data Analysis
news.describe()

,Category,News
count,705,705
unique,2,636
top,ca,The record date has been fixed at May 28.Inven...
freq,429,4


In [24]:
#Exploratory Data Analysis
news.groupby('Category').describe()

News                                                               
         count unique                                                top freq
Category                                                                     
ca         429    411  The record date has been fixed at May 28.Inven...    4
notca      276    225  The watchdog carried out an investigation of t...    2

In [25]:
import string
from nltk.corpus import stopwords

In [26]:
#Function to clean the text and pre processing it
def text_process(text):
    #Remove the punctuation
    #Remove the stopwords
    #Return list of clean text
    text=''.join([char for char in text if char not in string.punctuation])
    return [word for word in text.split() if word.lower() not in stopwords.words('english')]

In [27]:
#Verifying if the text processing is working
news['News'].head(5).apply(text_process)

0    [company, put, aggressive, bid, Rs, 6000, cror...
1    [Reliance, Capital, today, announced, appointm...
2    [Reports, emerged, Deloitte, statutory, audito...
3    [HG, Infra, Engineering, today, fixed, price, ...
4    [US, private, equity, firm, Blackstone, Group,...
Name: News, dtype: object

In [28]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [29]:
#Setting up the pipeline
pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=text_process)),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', MultinomialNB()),  # train on TF-IDF vectors w/ Naive Bayes classifier
])

In [30]:
#Training and Test split
news_train,news_test,category_train,category_test=train_test_split(news['News'],news['Category'],test_size=0.4)
print(len(news_train), len(news_test), len(news_train) + len(news_test))

423 282 705


In [31]:
pipeline.fit(news_train,category_train)

Pipeline(memory=None,
         steps=[('bow',
                 CountVectorizer(analyzer=<function text_process at 0x7f7f19a73050>,
                                 binary=False, decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('classifier',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [32]:
type(news_test)

pandas.core.series.Series

In [33]:
predictions = pipeline.predict(pd.Series('The government may soon ban PlayerUnknowns Battlegrounds (PUBG) in India. Reports suggesting a ban on 275 apps have started to surface online a month after the Indian government banned 59 Chinese apps, including popular ones like TikTok, UC Browser, citing data security concerns.'))

In [34]:
print(predictions)

['notca']


In [35]:
predictions = pipeline.predict(pd.Series('The board of Pfizer on Monday declared a special dividend of ₹320 a share of ₹10 each (3,200 per cent ) for the financial year ended March 31, 2020. The dividend will be paid to the eligible members of the company (whose names appear in the register of members as on the record date May 8) electronically through bank transfer on May 19.'))

In [36]:
print(predictions)

['ca']


In [37]:
predictions=pipeline.predict(news_test)

In [38]:
from sklearn.metrics import classification_report
print(classification_report(category_test,predictions,))

              precision    recall  f1-score   support

          ca       0.88      0.99      0.93       165
       notca       0.99      0.80      0.89       117

    accuracy                           0.91       282
   macro avg       0.93      0.90      0.91       282
weighted avg       0.92      0.91      0.91       282

